In [1]:
import pandas as pd
import numpy as np

from itertools import groupby
import lightgbm as lgb
import gc
from itertools import combinations
import plotly.express as px
from sklearn.metrics import mean_absolute_error
import numba
from numba import jit, njit, prange
import time

import logging

import warnings
warnings.filterwarnings("ignore")
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
# type the log background info
experiment = 132
experiment_description = "exp131 + normalized reference_price"
logging.basicConfig(filename='../logs/experiment.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filemode='a')

In [3]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """

    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    return df

In [4]:
train = pd.read_csv('../data/train.csv')
# revealed_targets = pd.read_csv('../data/revealed_targets.csv')
# test = pd.read_csv('../data/test.csv')
# sample_submission = pd.read_csv('../data/sample_submission.csv')
train.dropna(subset=['target'], inplace=True)
train['bucket'] = train['date_id']//97 # used for cv

# try the stock info
# stock_info = pd.read_csv('../data/sectornames_and_marketcap.csv')[['stock_id', 'sectorname']]

# for col in ['sectorname']:
#     le = LabelEncoder()
#     stock_info[col] = le.fit_transform(stock_info[col])

# train = train.merge(stock_info, on='stock_id', how='left')

train = reduce_mem_usage(train)

In [5]:
train.isnull().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 132
imbalance_buy_sell_flag          0
reference_price                132
matched_size                   132
far_price                  2894254
near_price                 2857092
bid_price                      132
bid_size                         0
ask_price                      132
ask_size                         0
wap                            132
target                           0
time_id                          0
row_id                           0
bucket                           0
dtype: int64

In [6]:
def match_weights(df):
    weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
    ]
    match_stock = {}
    for idx, w in enumerate(weights):
        match_stock[idx] = w
    
    df['weight_type'] = df['stock_id'].map(match_stock)
    return df

# cutoff info on seconds_in_bucket
def cut_off(seconds_in_bucket):
    if seconds_in_bucket <= 300:
        return 1
    elif seconds_in_bucket <= 480:
        return 2
    else:
        return 3

train = match_weights(train)
train['cut_off_time'] = train['seconds_in_bucket'].apply(cut_off)

stock_pca = pd.read_csv('../data/principal_components.csv')[['stock_id', 'PC1', 'PC2', 'PC3']]

train = train.merge(stock_pca, on=['stock_id'], how='left')

In [7]:
# third step
def global_features(df):
    """
    TODO: size/median_sizes; useful features' ratio;
    first_sizes and last_sizes looks wired -> can try to remove them; If they are useful, can consider to 
    use daily level to create the difference between them;
    imbalance_momentum can be update to use multiple windows -> check the difference with the rolling features;
    referene_prices over the middle of bid price and ask price;
    If 5 mins features are useful, then we can try to replicate the useful features we have before, same to the cutoff;
    Whether independent features are better than sum features -> test
    """
    global_stock_id_feats = {
        # size related features
        "median_size": df.groupby("stock_id")["bid_size"].median() + df.groupby("stock_id")["ask_size"].median(),
        "std_size": df.groupby("stock_id")["bid_size"].std() + df.groupby("stock_id")["ask_size"].std(),
        "max_sizes": df.groupby('stock_id')['bid_size'].max() + df.groupby('stock_id')['ask_size'].max(),
        "ptp_size": df.groupby("stock_id")["bid_size"].max() - df.groupby("stock_id")["bid_size"].min(),
        "mean_sizes": df.groupby('stock_id')['bid_size'].mean() + df.groupby('stock_id')['ask_size'].mean(),
        "ptp_IQR": df.groupby("stock_id")["bid_size"].quantile(0.75) - df.groupby("stock_id")["ask_size"].quantile(0.25),
        
        # price related features
        "median_price": df.groupby("stock_id")["bid_price"].median() + df.groupby("stock_id")["ask_price"].median(),
        "std_price": df.groupby("stock_id")["bid_price"].std() + df.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df.groupby("stock_id")["bid_price"].max() - df.groupby("stock_id")["ask_price"].min(),
        
        # 5 mins behavior
        "last5min_medvol": df[df.seconds_in_bucket>=300].groupby("stock_id")['bid_size'].median() + df[df.seconds_in_bucket>=300].groupby("stock_id")['ask_size'].median(),
        "first5min_medvol": df[df.seconds_in_bucket<300].groupby("stock_id")['bid_size'].median() + df[df.seconds_in_bucket<300].groupby("stock_id")['ask_size'].median(),
        
        # cut-off related features
        "cutoff_mean_sizes": df.groupby(['stock_id', 'cut_off_time'])['bid_size'].mean() + df.groupby(['stock_id', 'cut_off_time'])['ask_size'].mean(),
        "cutoff_imb_ratios": df.groupby(['stock_id', 'cut_off_time'])['matched_size'].mean() / df.groupby(['stock_id', 'cut_off_time'])['imbalance_size'].mean(),
    }

    return global_stock_id_feats

In [8]:
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [9]:
# first step
def feature_eng(df):
    """
    TODO: price_pressure can apply with current imbalance_szie with spread_intensity -> this direction
    can think deeper -> catching the imb info; 
    depth_pressure idea can be extended to (imbalance_size - matched_size) * (reference_price - mid_price);
    matched_momentum like imbalance_momentum; 
    far_price-near_price;
    windows size adjustment;
    the windows part, can we add weight type group by info;
    """
    
    df["dow"] = df["date_id"] % 5
    df["seconds"] = df["seconds_in_bucket"] % 60
    df["minute"] = df["seconds_in_bucket"] // 60   
    
    df["volume"] = df.eval("ask_size + bid_size")
    df['bid_ask_volume_diff'] = df.eval("ask_size - bid_size")
    df["liquidity_imbalance"] = df['bid_ask_volume_diff']/df["volume"]
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df['imbalance_ratio'] = df['imbalance_size'] / (1+df['matched_size'])
    
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * df["price_spread"]
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    #供需市场的差额
    df['depth_pressure'] = df['bid_ask_volume_diff'] * (df['far_price'] - df['near_price'])
    
    df["matched_momentum"] = df.groupby(['stock_id'])['matched_size'].diff(periods=1) / df['imbalance_size']  
    
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"] 
    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    
    df['near_ratio'] = df['near_price'] / df['reference_price']
    df['far_ratio'] = df['far_price'] / df['reference_price']
    df['near_size'] = df['near_ratio'] * df['matched_size']
    df['far_size'] = df['far_ratio'] * df['matched_size']

    # use the triplet method
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
        
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby(['stock_id', 'date_id'])[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby(['stock_id', 'date_id'])[col].pct_change(window)
            df[f"wt_{col}_ret_{window}"] = df.groupby(['weight_type', 'date_id'])[col].pct_change(window)
            
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'market_urgency', 'imbalance_momentum', 'size_imbalance']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_diff_{window}"] = df.groupby(['stock_id', 'date_id'])[col].diff(window)
    
    # one hot encoding
    df['cut_off_time_2'] = 0
    df['cut_off_time_3'] = 0
    df.loc[df["cut_off_time"] == 2, "cut_off_time_2"] = 1
    df.loc[df["cut_off_time"] == 3, "cut_off_time_3"] = 1
    # df.drop(columns=['cut_off_time'], inplace=True)
    df['far_near_diff'] = df['far_price'] - df['near_price']
    
    gc.collect()
    
    return df 
    
#     # useful for the last fold
#     df['all_prices_mean_std'] = df['all_prices_mean']/(1+df['all_prices_std'])
#     df['all_sizes_mean_std'] = df['all_sizes_mean']/(1+df['all_sizes_std'])
#     df['all_prices_skew_kurt'] = df['all_prices_skew']/(1+df['all_prices_kurt'])
#     df['all_sizes_skew_kurt'] = df['all_sizes_skew']/(1+df['all_sizes_kurt'])


In [10]:
# def std_group(value, mean, std):
#     if value >= mean and value <= mean+0.5*std:
#         return 1
#     elif value >= mean-0.5*std and value < mean:
#         return -1
#     elif value > mean+0.5*std and value <= mean + std:
#         return 2
#     elif value < mean-0.5*std and value >= mean - std:
#         return -2
#     elif value > mean + std and value <= mean + 2*std:
#         return 3
#     elif value < mean - std and value >= mean - 2*std:
#         return -3
#     elif value > mean + 2*std and value <= mean + 3*std:
#         return 4
#     elif value < mean - 2*std and value >= mean - 3*std:
#         return -4
#     elif value > mean + 3*std:
#         return 5
#     else:
#         return -5

In [11]:
# second step: very heavy calculation; should make the decision on keeping which part
@jit(numba.float64[:, :](numba.float64[:, :]))
def rolling_features(df):
    """
    TODO: Use long window to replace the cum ops or training with cum ops but inference with using
    the recorded value from the previous seconds in the same day;
    With the considerations about using long window to repalce cum, maybe we can add windows options in
    the rolling part and add std, skew, kurt ops here;
    for the last 10 sec wap part, we can calculate the pct change ratio between wap and ind_wap;
    ema part for near and far price;
    should rolling all use min_periods=1 -> reduce the null ratio;
    use rsi instead of rti;
    rolling features use some features from first step;
    buy_flag_diff can be optimized with calcualte the diff first then times the flag, 
    can also add skew and kurt into this part, some methods to calcuale the angle between two slope;
    approx slope for wap, matched_size, imbalanced_size and so on;
    can we avoid reset_index to save the memory;
    1mins level wap and industries related features: add median and max, the difference between max and min
    
    
    """
    ### Light Ops ###
    # last 10 sec wap and the ratio between wap and the mean of previous wap in the same weight type
    df['previous_wap'] = df.groupby(['stock_id', 'date_id'])['wap'].shift(1)
    df['previous_wt_wap_mean'] = df.groupby(['weight_type', 'date_id', 'seconds_in_bucket'])['previous_wap'].transform('mean')
    df['wap_percent_ind'] = df['wap'] / df['previous_wt_wap_mean']
    
    # last 10 sec median match size and im size for each weight type and the ratio between im size and match size
    df['previous_im_size'] = df.groupby(['stock_id', 'date_id'])['imbalance_size'].shift(1)
    df['previous_m_size'] = df.groupby(['stock_id', 'date_id'])['matched_size'].shift(1)
    df['previous_wt_im_median'] = df.groupby(['weight_type', 'date_id', 'seconds_in_bucket'])['previous_im_size'].transform('median')
    df['previous_wt_m_median'] = df.groupby(['weight_type', 'date_id', 'seconds_in_bucket'])['previous_m_size'].transform('median')
    df['previous_imbalance_ratio_percent_ind'] = df['imbalance_size'] / (1+df['matched_size'])/ df['previous_wt_im_median'] / (1+df['previous_wt_m_median'])
    # drop at the end
    # df.drop(columns=['previous_im_size', 'previous_m_size', 'previous_wt_im_median', 'previous_wt_m_median'], inplace=True)
    
    # ema
    alpha = 0.285
    beta = 1-alpha
    for col in ['matched_size', 'wap', 'imbalance_size']:
        df[f'ema_{col}'] = df[col]*alpha + df.groupby(['stock_id', 'date_id'])[col].shift(1)*alpha*beta + \
        df.groupby(['stock_id', 'date_id'])[col].shift(2)*alpha*beta**2 + df.groupby(['stock_id', 'date_id'])[col].shift(3)*alpha*beta**3 + \
        df.groupby(['stock_id', 'date_id'])[col].shift(4)*alpha*beta**4 + df.groupby(['stock_id', 'date_id'])[col].shift(5)*alpha*beta**5 + \
        df.groupby(['stock_id', 'date_id'])[col].shift(6)*alpha*beta**6
        
        for window in [3,5,7]:
            df[f'rolling_{window}_{col}_mean'] = df.groupby(['stock_id', 'date_id'])[col].rolling(window=window).mean().reset_index(level=[0, 1], drop=True)
    
    # get current index wap and previous 60 sec index wap and wap
    # then get the ratio between current and previous 60 sec; also get the ratio between wap and iwap in current status and previous 60 sec
    df['previous_1min_wap'] = df.groupby(['stock_id', 'date_id'])['wap'].shift(6)
    df['1min_wap_ratio'] = df['wap'] / df['previous_1min_wap']
    df['iwap'] = df['wap'] * df['weight_type']
    iwap = df.groupby(['date_id', 'seconds_in_bucket']).agg(index_wap=('iwap', 'sum'), total_weight=('weight_type', 'sum')).reset_index()
    iwap.sort_values(by=['date_id', 'seconds_in_bucket'], ascending=True, inplace=True)
    iwap['previous_1min_iwap'] = iwap.groupby(['date_id'])['index_wap'].shift(6)
    iwap['index_wap'] = iwap['index_wap']/iwap['total_weight']
    iwap['previous_1min_iwap'] = iwap['previous_1min_iwap']/iwap['total_weight']
    df = df.merge(iwap[['date_id', 'seconds_in_bucket', 'index_wap', 'previous_1min_iwap']], on=['date_id', 'seconds_in_bucket'], how='left')
    df['1min_iwap_ratio'] = df['index_wap'] / df['previous_1min_iwap']
    df['wap_index_ratio'] = df['wap']/df['index_wap']
    df['1min_wap_index_ratio'] = df['previous_1min_wap']/df['previous_1min_iwap']
    # drop at the end
    # df.drop(columns=['index_wap'], inplace=True)
    
    # Replace 'df' with your DataFrame and 'time_id', 'wap', 'imbalance_buy_sell_flag' with your column names
    df['norm_wap'] = df.groupby("time_id")['wap'].transform('mean')
    df['norm_wap'] = df['wap'] - df['norm_wap']
    df['norm_imbalance_buy_sell_flag'] = df.groupby("time_id")['imbalance_buy_sell_flag'].transform('mean')
    df['norm_imbalance_buy_sell_flag'] = df['imbalance_buy_sell_flag'] - df['norm_imbalance_buy_sell_flag']

    
    # 浪起来写fe
    for col in ['far_price', 'near_price',]:
#     ['imbalance_size', 'reference_price', 'matched_size', 'bid_price', 'bid_size', \
#                'ask_price', 'ask_size', 'wap']: #'far_price', 'near_price', 
        df['previous_1_min'] = df.groupby(['stock_id', 'date_id'])[col].shift(6)
        df[f'{col}_1min_approx_slope'] = df[col] - df['previous_1_min']
        df[f'{col}_wt_1min_approx_slope_mean'] = df.groupby(['weight_type', 'date_id', 'seconds_in_bucket'])[f'{col}_1min_approx_slope'].transform('mean')
        df[f'{col}_wt_1min_approx_slope_std'] = df.groupby(['weight_type', 'date_id', 'seconds_in_bucket'])[f'{col}_1min_approx_slope'].transform('std')
        df[f'{col}_wt_1min_approx_slope_min'] = df.groupby(['weight_type', 'date_id', 'seconds_in_bucket'])[f'{col}_1min_approx_slope'].transform('min')
        df[f'{col}_wt_1min_approx_slope_max'] = df.groupby(['weight_type', 'date_id', 'seconds_in_bucket'])[f'{col}_1min_approx_slope'].transform('max')
        df[f'{col}_1min_aslope_norm'] = (df[f'{col}_1min_approx_slope']-df[f'{col}_wt_1min_approx_slope_min'])/(1+df[f'{col}_wt_1min_approx_slope_max']-df[f'{col}_wt_1min_approx_slope_min'])
        df[f'{col}_wt_1min_approx_slope_mean_std'] = df[f'{col}_wt_1min_approx_slope_mean']/df[f'{col}_wt_1min_approx_slope_std']
    df['fp_np_aslope_ratio'] = df['far_price_1min_approx_slope']/df['near_price_1min_approx_slope']
#     df['rp_ms_aslope_ratio'] = df['reference_price_1min_approx_slope']/df['matched_size_1min_approx_slope']
#     df['rp_im_aslope_ratio'] = df['reference_price_1min_approx_slope']/df['imbalance_size_1min_approx_slope']
#     df['im_ms_aslope_ratio'] = df['imbalance_size_1min_approx_slope']/df['matched_size_1min_approx_slope']
#     df['ap_bp_aslope_ratio'] = df['ask_price_1min_approx_slope']/df['bid_price_1min_approx_slope']
#     df['ap_bs_aslope_ratio'] = df['ask_price_1min_approx_slope']/df['bid_size_1min_approx_slope']
#     df['bp_as_aslope_ratio'] = df['bid_price_1min_approx_slope']/df['ask_size_1min_approx_slope']
#     df['fp_rp_aslope_ratio'] = df['far_price_1min_approx_slope']/df['reference_price_1min_approx_slope']
#     df['np_rp_aslope_ratio'] = df['near_price_1min_approx_slope']/df['reference_price_1min_approx_slope']
#     df['wap_im_ms_aslope_ratio'] = df['wap_1min_approx_slope']/(df['matched_size_1min_approx_slope']*df['imbalance_size_1min_approx_slope'])
   
    
    ### Medium Ops ###
    # rolling median with win=6 for wap, adjust im_size, reference_price
    df['directed_imbalance_size'] = df['imbalance_size']*df['imbalance_buy_sell_flag']
    for col in ['wap', 'directed_imbalance_size', 'reference_price']:
        df[f'rolling_{col}_median'] = df.groupby(['stock_id', 'date_id'])[col].rolling(window=6).median().reset_index(level=[0, 1], drop=True)
        df[f'rolling_{col}_ratio'] = df[col] / df[f'rolling_{col}_median']
    
    # rolling sum in the 60 secs of (matched_size and im_size) & (bid_size, ask_size)
    for (col1, col2) in [('matched_size', 'imbalance_size'), ('bid_size', 'ask_size')]:
        df[f'roll_60_sum_{col1}']=df.groupby(['stock_id', 'date_id'])[col1].rolling(window=6).sum().reset_index(level=[0,1], drop=True)
        df[f'roll_60_sum_{col2}']=df.groupby(['stock_id', 'date_id'])[col2].rolling(window=6).sum().reset_index(level=[0,1], drop=True)
    # the ratio between match size and im size
    df['roll_60_sum_ms_im_ratio'] = df['roll_60_sum_matched_size']/(1+df['roll_60_sum_imbalance_size'])*df['imbalance_buy_sell_flag']
    # rolling 60 sec sum wap
    df['roll_60_sum_wap'] = (df['roll_60_sum_ask_size'] * df['bid_price'] + df['roll_60_sum_bid_size'] * df['ask_price'])/(df['roll_60_sum_bid_size']+df['roll_60_sum_ask_size']) 
    
    # approx slope for bid price, ask price, bid size and ask size
    df['buy_flag'] = np.where(df['imbalance_buy_sell_flag'] >= 0, 1, np.nan)
    df['sell_flag'] = np.where(df['imbalance_buy_sell_flag'] <= 0, 1, np.nan)
    df['buy_time'] = df['buy_flag'] * df['seconds_in_bucket']
    df['sell_time'] = df['sell_flag'] * df['seconds_in_bucket']
    
    for col in ['buy_time', 'sell_time']:
        df[f'{col}_start'] = df.groupby(['stock_id', 'date_id'])[col].rolling(window=6, min_periods=1).min().reset_index(level=[0,1], drop=True)
        df[f'{col}_end'] = df.groupby(['stock_id', 'date_id'])[col].rolling(window=6, min_periods=1).max().reset_index(level=[0,1], drop=True)
        df[f'{col}_diff'] = df[f'{col}_end'] - df[f'{col}_start']
    
    for (buy, sell) in [('bid_price', 'ask_price'), ('bid_size', 'ask_size')]:
        df['buy_site_flag'] = df['buy_flag'] * df[buy]
        df['sell_site_flag'] = df['sell_flag'] * df[sell]
        df['buy_flag_diff'] = df.groupby(['stock_id', 'date_id'])['buy_site_flag'].diff()
        df['sell_flag_diff'] = df.groupby(['stock_id', 'date_id'])['sell_site_flag'].diff()
        # calculate the sum of diff
        df['buy_site_diff_sum'] = df.groupby(['stock_id', 'date_id'])[f'buy_flag_diff'].rolling(window=7, min_periods=1).sum().reset_index(level=[0,1], drop=True)
        df[f'{buy}_diff_rolling_std'] = df.groupby(['stock_id', 'date_id'])[f'buy_flag_diff'].rolling(window=7, min_periods=1).std().reset_index(level=[0,1], drop=True)
        df['sell_site_diff_sum'] = df.groupby(['stock_id', 'date_id'])[f'sell_flag_diff'].rolling(window=7, min_periods=1).sum().reset_index(level=[0,1], drop=True)
        df[f'{sell}_diff_rolling_std'] = df.groupby(['stock_id', 'date_id'])[f'sell_flag_diff'].rolling(window=7, min_periods=1).std().reset_index(level=[0,1], drop=True)
        # calculate the approx slope
        df[f'{buy}_slope'] = df['buy_site_diff_sum']/ np.where(df['buy_time_diff'] == 0, np.nan, df['buy_time_diff'])
        df[f'{sell}_slope'] = df['sell_site_diff_sum']/ np.where(df['sell_time_diff'] == 0, np.nan, df['sell_time_diff'])
        # drop at the end
        # df.drop(columns=['buy_site_flag', 'sell_site_flag','buy_flag_diff', 'sell_flag_diff', 'buy_site_diff_sum', 'sell_site_diff_sum'], inplace=True)
    # drop at the end
#     df.drop(columns=['buy_flag', 'sell_flag', 'buy_time', 'sell_time', 'buy_time_start', 'buy_time_end', \
#                      'sell_time_start', 'sell_time_end', 'buy_time_diff', 'sell_time_diff'], inplace=True)
    
    # rsi
    for col in ['wap']:
        df[f'{col}_diff'] = df.groupby(['stock_id', 'date_id'])[col].diff()
        df[f'{col}_gain'] = df[f'{col}_diff'].where(df[f'{col}_diff'] > 0, 0)
        df[f'{col}_loss'] = -df[f'{col}_diff'].where(df[f'{col}_diff'] < 0, 0)
        for window in [7]:
            df[f'{col}_avg_gain_{window}'] = df[f'{col}_gain'].rolling(window=window).mean()
            df[f'{col}_avg_loss_{window}'] = df[f'{col}_loss'].rolling(window=window).mean()
            df[f'{col}_rs_{window}'] = df[f'{col}_avg_gain_{window}']/df[f'{col}_avg_loss_{window}']
            df[f'{col}_rsi_{window}'] = 100 - (100/(1+df[f'{col}_rs_{window}']))
            
            df.drop(columns=[f'{col}_avg_gain_{window}', f'{col}_avg_loss_{window}',\
                            f'{col}_rs_{window}'], inplace=True)
        df.drop(columns=[f'{col}_diff', f'{col}_gain', f'{col}_loss',], inplace=True)
            

    # approx ratio for reference price diff
    for col in ['reference_price']:
        df[f'{col}_diff'] = df.groupby(['stock_id', 'date_id'])[col].diff()
        df[f'{col}_diff_sum'] = df.groupby(['stock_id', 'date_id'])[f'{col}_diff'].rolling(window=7, min_periods=1).sum().reset_index(level=[0,1], drop=True)
        df[f'{col}_diff_rolling_std'] = df.groupby(['stock_id', 'date_id'])[f'{col}_diff_sum'].rolling(window=7, min_periods=1).std().reset_index(level=[0,1], drop=True)
        df[f'{col}_diff_ratio'] = df[f'{col}_diff_sum']/np.where(df[f'{col}_diff'] == 0, np.nan, df[f'{col}_diff'])
    
    ### Heavy Ops ###
    # adjusted rti
    for col in ['matched_size', 'wap', 'reference_price']:
        df[f'cum_{col}_std'] = df.groupby(['stock_id', 'date_id'])[col].expanding().std().reset_index(level=[0,1], drop=True)
        df[f'upper_{col}'] = df[col] + df[f'cum_{col}_std']
        df[f'lower_{col}'] = df[col] - df[f'cum_{col}_std']
        df[f'rolling_upper_{col}_max'] = df.groupby(['stock_id', 'date_id'])[f'upper_{col}'].rolling(window=7).max().reset_index(level=[0,1], drop=True)
        df[f'rolling_upper_{col}_min'] = df.groupby(['stock_id', 'date_id'])[f'lower_{col}'].rolling(window=7).min().reset_index(level=[0,1], drop=True)
        df[f'rti_{col}'] = (df[col] - df[f'rolling_upper_{col}_min'])/(df[f'rolling_upper_{col}_max'] - df[f'rolling_upper_{col}_min'])
        # drop at the end
        # df.drop(columns=[f'upper_{col}', f'lower_{col}'], inplace=True)
    # drop cum_wap_std; drop at the end
    # df.drop(columns=['cum_wap_std'], inplace=True)   

    # 1mins level wap and industries related features -> try to lower the difficulties on predicting the target
    # todo: add median
    df['previous_target'] = df['1min_wap_ratio'] - df['1min_iwap_ratio']
    df[f'cum_previous_target_mean'] = df.groupby(['stock_id', 'date_id'])['previous_target'].expanding().mean().reset_index(level=[0,1], drop=True)
    df[f'cum_previous_target_std'] = df.groupby(['stock_id', 'date_id'])['previous_target'].expanding().std().reset_index(level=[0,1], drop=True)
    df[f'cum_previous_target_skew'] = df.groupby(['stock_id', 'date_id'])['previous_target'].expanding().skew().reset_index(level=[0, 1], drop=True)
    df[f'cum_previous_target_kurtosis'] = df.groupby(['stock_id', 'date_id'])['previous_target'].expanding().kurt().reset_index(level=[0, 1], drop=True)
    
    # cumulative min, max and min max standard of the reference price, matched_size, imbalance_size
    for col in ['reference_price', 'matched_size', 'imbalance_size']:
        df[f'cum_{col}_min'] = df.groupby(['stock_id', 'date_id'])[col].expanding().min().reset_index(level=[0,1], drop=True)
        df[f'cum_{col}_max'] = df.groupby(['stock_id', 'date_id'])[col].expanding().max().reset_index(level=[0,1], drop=True)
        df[f'cum_{col}_max_min'] = (df[col] - df[f'cum_{col}_min'])/(1+df[f'cum_{col}_max'] - df[f'cum_{col}_min'])

        
    # drop at the end
    # df.drop(columns=['cum_reference_price_min', 'cum_imbalance_size_min'], inplace=True)
        
    # pct_change for reference_price and wap and get the skew and kurt
    for col in ['reference_price', 'wap',]:
        df[f'{col}_pct_change'] = df.groupby(['stock_id', 'date_id'])[col].pct_change()
        df[f'cum_{col}_skew'] = df.groupby(['stock_id', 'date_id'])[f'{col}_pct_change'].expanding().skew().reset_index(level=[0, 1], drop=True)
        df[f'cum_{col}_kurtosis'] = df.groupby(['stock_id', 'date_id'])[f'{col}_pct_change'].expanding().kurt().reset_index(level=[0, 1], drop=True)
        
        # vol features
        df[f'abs_return'] = df[f'{col}_pct_change'].abs()
        df[f'{col}_illiq'] = (df['abs_return'] / df['matched_size']).rolling(10).mean()
        
        df['neg_retrun_flag'] = np.where(df[f'{col}_pct_change']<0, 1, 0)
        df['neg_retrun_flag'] = df['neg_retrun_flag'] * df['abs_return']
        df[f'{col}_negative_returns_illiq'] = df['neg_retrun_flag'].rolling(10).sum() / (1+(df['imbalance_size']/df['matched_size']).rolling(10).sum())
        
    # drop at the end
    # df.drop(columns=['cum_wap_skew', 'reference_price_pct_change'], inplace=True)
   
    df.drop(columns=['previous_im_size', 'previous_m_size', 'previous_wt_im_median', 'previous_wt_m_median', \
                    'upper_matched_size', 'lower_matched_size', 'upper_wap', 'lower_wap', \
                    'upper_reference_price', 'lower_reference_price', 'buy_site_flag', 'sell_site_flag', \
                    'buy_flag_diff', 'sell_flag_diff', 'buy_site_diff_sum', 'sell_site_diff_sum', \
                    'buy_flag', 'sell_flag', 'buy_time', 'sell_time', 'buy_time_start', 'buy_time_end', \
                    'sell_time_start', 'sell_time_end', 'buy_time_diff', 'sell_time_diff', \
                    'cum_wap_skew', 'reference_price_diff', \
                    'cum_wap_std', 'cum_reference_price_min', 'cum_imbalance_size_min', 'index_wap', 'reference_price_pct_change',\
                    'abs_return', 'neg_retrun_flag', 'previous_1_min',\
                    ], inplace=True)

    return df
            
        
    
#     # rsi
#     df['gain'] = np.where(df['rp_diff']>0, df['rp_diff'], 0)
#     df['loss'] = np.where(df['rp_diff']<0, -df['rp_diff'], 0)
#     df['gain_flag'] = np.where(df['rp_diff']>0, 1, 0)
#     df['loss_flag'] = np.where(df['rp_diff']<0, 1, 0)
#     for window in [7,14]:
#         df[f'gain_{window}_sum'] = df.groupby(['stock_id', 'date_id'])[f'gain'].rolling(window=window).sum().reset_index(level=[0,1], drop=True)
#         df[f'loss_{window}_sum'] = df.groupby(['stock_id', 'date_id'])[f'loss'].rolling(window=window).sum().reset_index(level=[0,1], drop=True)
#         df[f'gain_flag_{window}_sum'] = df.groupby(['stock_id', 'date_id'])[f'gain_flag'].rolling(window=window).sum().reset_index(level=[0,1], drop=True)
#         df[f'loss_flag_{window}_sum'] = df.groupby(['stock_id', 'date_id'])[f'loss_flag'].rolling(window=window).sum().reset_index(level=[0,1], drop=True)
#         df[f'gain_{window}_mean'] = df[f'gain_{window}_sum']/df[f'gain_flag_{window}_sum']
#         df[f'loss_{window}_mean'] = df[f'loss_{window}_sum']/df[f'loss_flag_{window}_sum']
#         df[f'relative_{window}_strength'] = df[f'gain_{window}_mean']/df[f'loss_{window}_mean']
#         df[f'rsi__{window}'] = 1/(1+df[f'relative_{window}_strength'])
#         'gain', 'loss', 'gain_flag', 'loss_flag', 'relative_7_strength', 'relative_14_strength', \
#                     'gain_7_sum', 'gain_flag_7_sum', 'gain_14_sum', 'gain_14_sum', \
#                     'loss_7_sum', 'loss_flag_7_sum', 'loss_14_sum', 'loss_flag_14_sum',


In [12]:
# put all features engineering func together
def generate_all_features1(df):
    df = feature_eng(df)
    df = rolling_features(df)
    return df

def generate_all_features2(df):
    for key, value in global_stock_id_feats.items():
        if 'cut_off' not in key:
            df[f"global_{key}"] = df["stock_id"].map(value.to_dict())
        else:
            df[f"global_{key}"] = df["stock_id", 'cut_off_time'].map(value.to_dict())

    # flag features
    df['high_volume'] = np.where(df['volume'] > df['global_median_size'], 1, 0)
    cols = [c for c in df.columns if c not in ['row_id', 'date_id','time_id', 'cut_off_time', 'stock_id']]
    df = df[cols]
    return df

In [13]:
def zero_sum(prices, volumes):
    
#    I got this idea from https://github.com/gotoConversion/goto_conversion/
    
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out

In [14]:
start = time.time()

lgb_params = {
        "objective" : "mae",
        "n_estimators" : 3000,
        "num_leaves" : 128,
        "subsample" : 0.6,
        "colsample_bytree" : 0.6,
        "learning_rate" : 0.05,
        "n_jobs" : 4,
        # "device" : "gpu",
        "verbosity": -1,
        "importance_type" : "gain",
    }

model = lgb.LGBMRegressor(**lgb_params)

# model = lgb.LGBMRegressor(learning_rate=0.065, max_depth=6, n_estimators=800,
#               num_leaves=32, objective='mae', random_state=13, metric_freq=50, 
#               reg_alpha=0.01, reg_lambda=3.25, colsample_bytree=0.7, subsample=0.65)

# create rolling features which can be apply on the whole dataset -> important: online should be redefine this part of code
train.sort_values(by=['stock_id', 'date_id', 'seconds_in_bucket'], ascending=True, inplace=True)

cv_mae = 0
cv_mae_goto = 0
y_min, y_max = -64, 64
change_cv_mae = 0

cv_folds = [([0,1,4],2,3),([0,3,4],1,2),([2,3,4],0,1),([1,2,3],4,0), ([0,1,2],3,4)]

for i in range(5):
    print(f"Working on fold {i+1}")
    train_bucket, val_bucket, test_bucket = cv_folds[i]
    
    test_ = train[train['bucket']==test_bucket]
    train_val = train[train['bucket']!=test_bucket]
    
    # then the group features will be only calculated on train and val, not on test
    train_val = generate_all_features1(train_val)
    test_ = generate_all_features1(test_)
    global_stock_id_feats = global_features(train_val)
    train_val = generate_all_features2(train_val)
    test_ = generate_all_features2(test_)
    
    train_val = reduce_mem_usage(train_val)
    test_ = reduce_mem_usage(test_)
    
    train_ = train_val[train_val['bucket'].isin(train_bucket)]
    val_ = train_val[train_val['bucket']==val_bucket]
    
    y_train, y_val, y_test = train_['target'], val_['target'], test_['target']
    X_train, X_val, X_test = train_.drop(columns=['target', 'bucket',]), val_.drop(columns=['target', 'bucket',]), test_.drop(columns=['target', 'bucket',])
    
    eval_set = [(X_val, y_val)]

    model.fit(X_train, y_train, eval_set=eval_set, eval_metric='mae', callbacks=[
            lgb.callback.early_stopping(stopping_rounds=50),
            lgb.callback.log_evaluation(period=100),
        ],)
    
    pred = model.predict(X_test)
    
    pred_goto = zero_sum(pred, X_test.loc[:,'bid_size'] + X_test.loc[:,'ask_size'])
    
    mae_pred = mean_absolute_error(y_test, pred)
    mae_pred_goto = mean_absolute_error(y_test, pred_goto)
    
    cv_mae += mae_pred
    cv_mae_goto += mae_pred_goto

    # new added method for testing -> train more steps as we used to do
#     infer_params = lgb_params.copy()
#     infer_params["n_estimators"] = int(1.2 * model.best_iteration_)
#     infer_lgb_model = lgb.LGBMRegressor(**infer_params)
#     infer_lgb_model.fit(X_train, y_train)
    
#     more_step_pred = infer_lgb_model.predict(X_test)
#     more_step_pred_goto = zero_sum(more_step_pred, X_test.loc[:,'bid_size'] + X_test.loc[:,'ask_size'])
    
#     more_step_mae_pred = mean_absolute_error(y_test, more_step_pred)
#     more_step_mae_pred_goto = mean_absolute_error(y_test, more_step_pred_goto)
    
#     more_step_cv_mae += more_step_mae_pred
#     more_step_cv_mae_goto += more_step_mae_pred_goto
    lgb_prediction = pred - np.mean(pred)
    clipped_predictions = np.clip(lgb_prediction, y_min, y_max)
    
    change_mae_pred = mean_absolute_error(y_test, clipped_predictions)
    change_cv_mae += change_mae_pred

    
print(f'CV mae: {cv_mae/5}')
print(f'CV mae goto: {cv_mae_goto/5}')
print(f'change CV mae: {change_cv_mae/5}')

end = time.time()
time_cost = (end - start)/60
print(f'the process run in {time_cost} mins')

Working on fold 1
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l1: 6.33187
[200]	valid_0's l1: 6.32182
[300]	valid_0's l1: 6.31985
Early stopping, best iteration is:
[317]	valid_0's l1: 6.31958
Working on fold 2
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l1: 7.00652
[200]	valid_0's l1: 6.99073
[300]	valid_0's l1: 6.98645
[400]	valid_0's l1: 6.98488
Early stopping, best iteration is:
[420]	valid_0's l1: 6.98463
Working on fold 3
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l1: 5.69056
[200]	valid_0's l1: 5.68069
[300]	valid_0's l1: 5.67844
[400]	valid_0's l1: 5.6775
[500]	valid_0's l1: 5.67689
Early stopping, best iteration is:
[471]	valid_0's l1: 5.6767
Working on fold 4
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l1: 5.92044
[200]	valid_0's l1: 5.91206
[300]	valid_0's l1: 5.91075
[400]	valid_0's l1: 5.91026
Early stopping, best iteration is:
[364]	valid_0'

In [15]:
print(f'number of features: {X_train.shape[1]}')

number of features: 258


In [16]:
logging.info(f"Experiment {experiment}: {experiment_description}")
logging.info(f'Experiment {experiment} mae: {cv_mae/5}')
logging.info(f'Experiment {experiment} mae_goto: {cv_mae_goto/5}')
logging.info(f'Experiment {experiment} time series mae: {mae_pred}')
logging.info(f'Experiment {experiment} change_mae: {change_cv_mae/5}')

In [ ]:
comments = '暂时维持不适用ye transoform之后再看有没有统一好用的'
logging.info(f"Experiment {experiment} comment: {comments}")